In [9]:
#IMPORTING scikit packages
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats


In [10]:
#IMPORTING SETUP PACKAGES
import pickle
import os.path
import os
import pandas as pd
import numpy as np

import psycopg2
import timeit
from datetime import datetime, timedelta, date
import datetime as dt
from calendar import monthrange
import random
import io
import boto3
import pyarrow as pa
import pyarrow.parquet as pq

from s3fs.core import S3FileSystem

import s3fs


%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 5000)

In [11]:
df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top_seller_YTD.pickle')

df

,item_code,update_date,item_price,delivery_weeks,sales,item_parent_item_code,item_main_category,item_sub_category_1,item_sub_category_2,log_price,log_delivery_weeks,log_sales
0,000000001000022708,2020-01-01,99.99,1,2,000000008000017295,MATRATZEN&LATTENROSTE,LATTENROSTE,FEDERHOLZ,4.615022,0.693147,1.098612
1,000000001000024281,2020-01-01,59.99,1,10,000000008000002127,MATRATZEN&LATTENROSTE,MATRATZEN,KALTSCHAUMMATRATZEN,4.110710,0.693147,2.397895
2,000000001000015944,2020-01-01,64.99,2,4,000000008000012450,SCHLAFZIMMERMÖBEL,NACHTKOMMODEN,NACHTKOMMODEN,4.189503,1.098612,1.609438
3,000000001000016020,2020-01-01,119.99,2,8,000000008000001083,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.795708,1.098612,2.197225
4,000000001000015958,2020-01-01,129.99,1,11,000000008000001069,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.875121,0.693147,2.484907
5,000000001000016021,2020-01-01,119.99,5,23,000000008000001083,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.795708,1.791759,3.178054
6,000000001000016019,2020-01-01,119.99,5,0,000000008000001083,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.795708,1.791759,0.000000
7,000000001000016133,2020-01-01,119.99,1,3,000000008000001083,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.795708,0.693147,1.386294
8,000000001000022573,2020-01-01,99.99,2,7,000000008000001979,MATRATZEN&LATTENROSTE,LATTENROSTE,FEDERHOLZ,4.615022,1.098612,2.079442
9,000000001000015959,2020-01-01,129.99,0,1,000000008000001069,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.875121,0.000000,0.693147


In [12]:
df.describe()

,item_price,delivery_weeks,sales,log_price,log_delivery_weeks,log_sales
count,3230.000000,3230.000000,3230.000000,3230.000000,3230.000000,3230.000000
mean,104.687214,1.357585,10.807740,4.608210,0.810876,2.228890
std,30.617754,0.926201,8.169269,0.341425,0.273162,0.742814
min,49.990000,0.000000,0.000000,3.931630,0.000000,0.000000
25%,64.990000,1.000000,5.000000,4.189503,0.693147,1.791759
50%,119.990000,1.000000,9.000000,4.795708,0.693147,2.302585
75%,119.990000,1.000000,14.000000,4.795708,0.693147,2.708050
max,139.990000,9.000000,68.000000,4.948689,2.302585,4.234107


In [13]:
reg_model_no_fe = smf.ols("log_sales ~ log_price", df).fit()
print(reg_model_no_fe.summary())


                            OLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     32.66
Date:                Wed, 02 Dec 2020   Prob (F-statistic):           1.20e-08
Time:                        19:33:38   Log-Likelihood:                -3606.1
No. Observations:                3230   AIC:                             7216.
Df Residuals:                    3228   BIC:                             7228.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2323      0.176     18.360      0.0

In [14]:
reg_model_with_fe = smf.ols("log_sales ~ log_price + C(item_code)", df).fit()
print(reg_model_with_fe.summary())

                            OLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                     118.2
Date:                Wed, 02 Dec 2020   Prob (F-statistic):          3.26e-210
Time:                        19:33:38   Log-Likelihood:                -3117.2
No. Observations:                3230   AIC:                             6256.
Df Residuals:                    3219   BIC:                             6323.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte